## Storage Model revisited

In [1]:
!pip install pulp
from pulp import *

In [2]:
# Creates a list of the products
months = ['000','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']

# A dictionary of the unit cost per month
cost = {'000': 0,
    'JAN': 240, 
         'FEB': 250,
       'MAR': 265, 
         'APR': 285,
       'MAY': 280, 
         'JUN': 260}

# A dictionary of the demand per month
demand = {'000': 0,
          'JAN': 1000, 
         'FEB': 4500,
       'MAR': 6000, 
         'APR': 5500,
       'MAY': 3500, 
         'JUN': 4000}

# A dictionary of the min production per month
minOut = {'000': 0,
          'JAN': 2000, 
         'FEB': 1750,
       'MAR': 2000, 
         'APR': 2250,
       'MAY': 2000, 
         'JUN': 1750}

# A dictionary of the max production per month
maxOut = {'JAN': 4000, 
         'FEB': 3500,
       'MAR': 4000, 
         'APR': 4500,
       'MAY': 4000, 
         'JUN': 3500,
         '000': 0}


In [3]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Upton Corporation Production Planning", LpMinimize)

# A dictionary of the output variables
o = LpVariable.dicts("output",months,0)

# A dictionary of the storage variables
s = LpVariable.dicts("storage",months,1500,6000)

# A dictionary of the overtime output variables
ot = LpVariable.dicts("overtimeOutput",months,0)

# A dictionary of the external supplier variables
sp = LpVariable.dicts("supplierPurchases",months,0)

# Create objective function
prob += lpSum([o[i]*cost[i] for i in months]) + lpSum(
    [0.015*cost[months[i+1]]*0.5*(s[months[i+1]]+s[months[i]]) for i in range(6)]) + lpSum(
    [ot[i]*cost[i]*1.1 for i in months]) + lpSum(
    [sp[i]*cost[i]*1.2 for i in months]), "Total Cost"

#Enforce values of init period
prob += s["000"] == 2750
prob += sp["000"] == 0

# Create constraints
for month in months:
    prob += o[month] >= minOut[month], "minOut{}".format(month)
    prob += o[month] <= maxOut[month], "maxOut{}".format(month)
    prob += ot[month] <= 0.2*maxOut[month], "maxOvertimeOut{}".format(month)

for i in range(6):
    prob += s[months[i]]+o[months[i+1]]+ot[months[i+1]]+sp[months[i+1]]-demand[months[i+1]]==s[months[i+1]]

In [4]:
print(prob)

Upton Corporation Production Planning:
MINIMIZE
285*output_APR + 250*output_FEB + 240*output_JAN + 260*output_JUN + 265*output_MAR + 280*output_MAY + 313.5*overtimeOutput_APR + 275.0*overtimeOutput_FEB + 264.0*overtimeOutput_JAN + 286.0*overtimeOutput_JUN + 291.5*overtimeOutput_MAR + 308.0*overtimeOutput_MAY + 1.7999999999999998*storage_000 + 4.2375*storage_APR + 3.8625*storage_FEB + 3.675*storage_JAN + 1.95*storage_JUN + 4.125*storage_MAR + 4.05*storage_MAY + 342.0*supplierPurchases_APR + 300.0*supplierPurchases_FEB + 288.0*supplierPurchases_JAN + 312.0*supplierPurchases_JUN + 318.0*supplierPurchases_MAR + 336.0*supplierPurchases_MAY + 0.0
SUBJECT TO
_C1: storage_000 = 2750

_C2: supplierPurchases_000 = 0

minOut000: output_000 >= 0

maxOut000: output_000 <= 0

maxOvertimeOut000: overtimeOutput_000 <= 0

minOutJAN: output_JAN >= 2000

maxOutJAN: output_JAN <= 4000

maxOvertimeOutJAN: overtimeOutput_JAN <= 800

minOutFEB: output_FEB >= 1750

maxOutFEB: output_FEB <= 3500

maxOvertimeOu

In [5]:
# Optimize

prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

Status = Optimal


In [8]:
# Print the value of the variables at the optimum
for i in months:
    print("%s | s= %f | o= %f | ot= %f | sp= %f" % (i, s[i].varValue, o[i].varValue, ot[i].varValue, sp[i].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

000 | s= 2750.000000 | o= 0.000000 | ot= 0.000000 | sp= 0.000000
JAN | s= 6000.000000 | o= 4000.000000 | ot= 250.000000 | sp= 0.000000
FEB | s= 5700.000000 | o= 3500.000000 | ot= 700.000000 | sp= 0.000000
MAR | s= 3700.000000 | o= 4000.000000 | ot= 0.000000 | sp= 0.000000
APR | s= 1500.000000 | o= 3300.000000 | ot= 0.000000 | sp= 0.000000
MAY | s= 2000.000000 | o= 4000.000000 | ot= 0.000000 | sp= 0.000000
JUN | s= 1500.000000 | o= 3500.000000 | ot= 0.000000 | sp= 0.000000
Objective = 6205660.000000
